<a href="https://colab.research.google.com/github/JUANCHOLOCO/LanGrandTaller1/blob/main/colabLaboratorio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
# Instalación
%%capture
# Instalar paquetes necesarios (ejecutar en celda separada en Colab)
!pip install -U langchain-google-genai langgraph langchain-core langchain-community python-dotenv

In [86]:
# Código: Agente LangGraph para dominio Java (ajustado)
# NOTA: instala dependencias en tu entorno antes de ejecutar.
import os
from typing import TypedDict, Annotated, Sequence, Optional, Dict
import operator
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

# LLM: imports (ajusta según librería exacta instalada)
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM: imports (ajusta según librería exacta instalada)
# from langchain_google_genai import ChatGoogleGenerativeAI

In [87]:
# -------------------------
# TypedDict (esquema de estado)
# -------------------------
class AgentState(TypedDict):
    messages: Annotated[Sequence[dict], operator.add]
    analysis_results: Dict[str, object]
    retries: int
    metadata: Optional[Dict[str, object]]


In [89]:

import os
# 2. INICIALIZAR AGENTES/LLMs
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
# -------------------------
# 🔥 CORRECCIÓN: Instancia real del modelo Gemini 2.5 Flash
# -------------------------
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.3,
    max_output_tokens=1000,
    google_api_key='GOOGLE_API_KEY'

)

In [90]:
# -------------------------
# LLM wrapper: intenta varios métodos de invocación para robustez
# -------------------------
def send_messages_to_llm(llm_instance, messages):
    """
    Intentará invocar el LLM con la lista 'messages' (lista de dicts role/content).
    Soporta varios métodos (invoke, __call__, generate, predict_messages).
    """
    # Si el LLM provee 'invoke' (tu versión original)
    try:
        if hasattr(llm_instance, "invoke"):
            return llm_instance.invoke(messages)
        # callable LLM (p.ej. langchain ChatModels suelen ser callable)
        if callable(llm_instance):
            return llm_instance(messages)
        # método 'generate' o 'predict_messages'
        if hasattr(llm_instance, "generate"):
            return llm_instance.generate(messages)
        if hasattr(llm_instance, "predict_messages"):
            return llm_instance.predict_messages(messages)
    except Exception as e:
        # No abortar; devolver una estructura coherente para manejo de fallback
        return type("LLMResponse", (), {"content": f"LLM invocation error: {e}"})

    raise RuntimeError("LLM instance no tiene método compatible (invoke/call/generate/predict_messages)")


In [91]:
# -------------------------
# Nodos del grafo
# -------------------------
def analyze_input_node(state: AgentState):
    """
    Analiza la entrada: llama al LLM con un prompt de 'analista experto'.
    Actualiza analysis_results y current_step.
    """
    messages = list(state["messages"])  # copy-safe
    system_prompt = {
        "role": "system",
        "content": (
            "Eres un analista experto en backend Java. Extrae intención (debug/config/review), "
            "confidence estimada y detalles (stacktrace, componentes involucrados)."
        )
    }
    gemini_messages = [system_prompt] + messages

    response = send_messages_to_llm(llm, gemini_messages)
    content = getattr(response, "content", str(response))

    # Aquí podrías parsear JSON si el LLM devuelve un JSON estructurado
    # Para el ejemplo, hacemos parseo simple heurístico:
    # (en producción: usar schema validation / tools como pydantic)
    analysis_results = {
        "intent": "unknown",
        "confidence": 0.5,
        "details": content
    }

    # heurística simple: si aparece 'JdbcCursorItemReader' considerarlo debug_spring_batch
    if "JdbcCursorItemReader" in content or "cursor" in content.lower():
        analysis_results["intent"] = "debug_spring_batch"
        analysis_results["confidence"] = 0.9
    elif "stacktrace" in content.lower() or "exception" in content.lower():
        analysis_results["intent"] = "debug"
        analysis_results["confidence"] = 0.7

    return {
        "analysis_results": analysis_results
    }

In [92]:
def decide_node(state: AgentState):
    """
    Nodo de decisión. Dependiendo de confidence y reglas, indica la siguiente acción.
    Retorna current_step pero también a través del grafo hacemos la transición condicional.
    """
    analysis = state["analysis_results"]
    confidence = float(analysis.get("confidence", 0.0))
    # umbral configurable: 0.70
    threshold = 0.7
    next_step = "respond" if confidence >= threshold else "fallback"
    return {"next_step_choice": next_step}

In [93]:
def generate_response_node(state: AgentState):
    """
    Genera la respuesta final (respuestas técnicas con snippets).
    Actualiza messages y current_step.
    """
    messages = list(state["messages"])
    analysis = state["analysis_results"]
    system_prompt = {
        "role": "system",
        "content": (
            f"Eres un asistente técnico. Basado en este análisis: intent={analysis.get('intent')}, "
            f"confidence={analysis.get('confidence')}. Proporciona respuesta técnica concisa y ejemplos."
        )
    }
    gemini_messages = [system_prompt] + messages
    response = send_messages_to_llm(llm, gemini_messages)
    content = getattr(response, "content", str(response))

    new_messages = messages + [{"role": "assistant", "content": content}]
    return {"messages": new_messages}


In [94]:
def fallback_node(state: AgentState):
    """
    Fallback: cuando confidence es baja o falta información. Incrementa retries y pide más datos.
    """
    messages = list(state["messages"])
    retries = int(state.get("retries", 0)) + 1
    fallback_msg = (
        "No tengo suficiente información confiable para dar un diagnóstico preciso. "
        "Por favor adjunta: logs (últimas 200 líneas), stacktrace completo, versión Java, "
        "configuración del DataSource y del ItemReader. Mientras tanto, revisa: enable DEBUG logs, "
        "thread dump y heap dump."
    )
    new_messages = messages + [{"role":"assistant","content":fallback_msg}]
    return {"messages": new_messages, "retries": retries}


In [95]:
# -------------------------
# Wrappers para el grafo (funciones que LangGraph espera)
# -------------------------
def analyze_wrap(state):
    return analyze_input_node(state)

def decide_wrap(state):
    return decide_node(state)

def respond_wrap(state):
    return generate_response_node(state)

def fallback_wrap(state):
    return fallback_node(state)


In [96]:
# -------------------------
# Construcción del grafo
# -------------------------
workflow = StateGraph(AgentState)
workflow.add_node("analyze", analyze_wrap)
workflow.add_node("decide", decide_wrap)
workflow.add_node("respond", respond_wrap)
workflow.add_node("fallback", fallback_wrap)

workflow.add_edge(START, "analyze")
workflow.add_edge("analyze", "decide")
# Edges from decide are handled por la lógica de decisión; aquí añadimos ambos caminos
workflow.add_edge("decide", "respond")
workflow.add_edge("decide", "fallback")
workflow.add_edge("respond", END)
workflow.add_edge("fallback", END)


In [97]:
# -------------------------
# Checkpointing y ejecución (ejemplo)
# -------------------------
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "langgraph_java_backend_agent_v1"}}
initial_state: AgentState = {
    "messages": [
        {"role": "user", "content": "Tengo este error org.springframework.dao.InvalidDataAccessResourceUsageException: Unexpected cursor position change. Uso JdbcCursorItemReader."}
    ],
    "current_step": "initial",
    "analysis_results": {},
    "retries": 0,
    "metadata": {}
}


In [98]:
def pretty_print_result(result):
    messages = result.get("messages", [])

    # Buscar el último mensaje del assistant
    assistant_msgs = [m for m in messages if m.get("role") == "assistant"]

    print("\n========== RESPUESTA FINAL ==========\n")

    if assistant_msgs:
        print(assistant_msgs[-1]["content"])
    else:
        print("⚠️ No se encontró respuesta del asistente.")

    print("\n=====================================\n")


In [99]:
def pretty_print_with_errors(result):
    print("\n========== RESULTADO ==========\n")

    # Respuesta del asistente
    assistant_msgs = [m for m in result.get("messages", []) if m.get("role") == "assistant"]

    if assistant_msgs:
        print("🧠 Respuesta del asistente:\n")
        print(assistant_msgs[-1]["content"])
    else:
        print("⚠️ No hubo respuesta del asistente.")

    # Error del LLM
    analysis = result.get("analysis_results", {})
    details = analysis.get("details")

    if details and "API key not valid" in details:
        print("\n🚨 ERROR DE CONFIGURACIÓN:")
        print("La API Key de Gemini no es válida o no está configurada.")

    print("\n===============================\n")



In [100]:
# Ejecuta el grafo (si llm está instanciado). Si no, el wrapper devolverá warning.
if llm is None:
    print("AVISO: llm es None. Instancia tu ChatGoogleGenerativeAI y asigna a 'llm' antes de invocar.")
else:
    result = app.invoke(initial_state, config=config)
    pretty_print_result(result)
    pretty_print_with_errors(result)
    print("Resultado final:", result)


========== RESPUESTA FINAL ==========

LLM invocation error: Error calling model 'gemini-2.5-flash' (INVALID_ARGUMENT): 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}



========== RESULTADO ==========

🧠 Respuesta del asistente:

LLM invocation error: Error calling model 'gemini-2.5-flash' (INVALID_ARGUMENT): 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'met